<h1 align="center"> Desafio de Engenharia de Dados</h1>
<br />

<img src="img/spark-logo.png" width="250" height="120" align="left" />
<img src="img/semantix-horizontal.png" width="500" height="100" align="right" />

 ## Descrição
 
Fonte oficial do dateset: http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html

Dados:
- Jul 01 to Jul 31, ASCII format, 20.7 MB gzip compressed, 205.2 MB.
- Aug 04 to Aug 31, ASCII format, 21.8 MB gzip compressed, 167.8 MB.

Sobre o dataset: Esses dois conjuntos de dados possuem todas as requisições HTTP para o servidor da NASA Kennedy Space Center WWW na Flórida para um período específico.

Os logs estão em arquivos ASCII com uma linha por requisição com as seguintes colunas:
- Host fazendo a requisição. Um hostname quando possível, caso contrário o endereço de internet se o nome não puder ser identificado.
- Timestamp no formato "DIA/MÊS/ANO:HH:MM:SS TIMEZONE"
- Requisição (entre aspas)
- Código do retorno HTTP
- Total de bytes retornados

<br /><br />
<hr />

Primeiro, as bibliotecas necessárias são importadas

In [1]:
import re
from datetime import datetime

Após, o download dos arquivos que contém os datasets são efetuados e ambos são descompactados

In [2]:
!wget --quiet ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz
!wget --quiet ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz
print("Data Downloaded!")

Data Downloaded!


In [3]:
!gunzip NASA_access_log_Jul95.gz
!gunzip NASA_access_log_Aug95.gz
print("Data Extracted!")

Data Extracted!


Depois, os datasets são carregados em RDDs, e estes são unidos para formar um único RDD

In [4]:
jul_rdd = sc.textFile('NASA_access_log_Jul95')
aug_rdd = sc.textFile('NASA_access_log_Aug95')

full_dataset_rdd = jul_rdd + aug_rdd

Após isso, os dados no RDD resultante são estruturados em tuplas de 4 posições em que a primeira contém o host emissor da requisição, a segunda contém a data da requisição, a terceira ontém o código de retorno da resposta e a quarta contém o número de bytes enviados ao cliente

In [5]:
# setting the structure to make it easier to query for data later on
def set_data_structure(line):
    host = re.search(r'(.*) - - ', line)
    host = host.group(1) if host else ''
    
    date = re.search(r'\[([0-9A-Za-z\/]*):.*\]', line)
    date = date.group(1) if date else ''
    
    status_code = re.search(r'\" ([0-9]+) ', line)
    status_code = int(status_code.group(1)) if status_code else 0
    
    bytes_transfered = re.search(r' ([0-9]+)$', line)
    bytes_transfered = int(bytes_transfered.group(1)) if bytes_transfered else 0
    
    return (host, date, status_code, bytes_transfered)


structured_rdd = full_dataset_rdd.map(lambda line: set_data_structure(line))
structured_rdd.cache()
structured_rdd.take(10)

[('199.72.81.55', '01/Jul/1995', 200, 6245),
 ('unicomp6.unicomp.net', '01/Jul/1995', 200, 3985),
 ('199.120.110.21', '01/Jul/1995', 200, 4085),
 ('burger.letters.com', '01/Jul/1995', 304, 0),
 ('199.120.110.21', '01/Jul/1995', 200, 4179),
 ('burger.letters.com', '01/Jul/1995', 304, 0),
 ('burger.letters.com', '01/Jul/1995', 200, 0),
 ('205.212.115.106', '01/Jul/1995', 200, 3985),
 ('d104.aa.net', '01/Jul/1995', 200, 3985),
 ('129.94.144.152', '01/Jul/1995', 200, 7074)]

## Questões

1) Número de hosts únicos.

In [6]:
unique_hosts = structured_rdd.map(lambda req: req[0]).distinct()
unique_hosts.count()

137979

2) O total de erros 404.

In [7]:
errors_404 = structured_rdd.filter(lambda req: req[2] == 404)
errors_404.cache()
errors_404.count()

20901

3) Os 5 URLs que mais causaram erro 404.

In [8]:
hosts_with_most_404 = errors_404.map(lambda req: (req[0], 1)).reduceByKey(lambda x, y: x + y)
hosts_with_most_404.takeOrdered(5, lambda req: -req[1])

[('hoohoo.ncsa.uiuc.edu', 251),
 ('piweba3y.prodigy.com', 157),
 ('jbiagioni.npt.nuwc.navy.mil', 132),
 ('piweba1y.prodigy.com', 114),
 ('www-d4.proxy.aol.com', 91)]

4) Quantidade de erros 404 por dia.

In [9]:
errors_404_per_day = errors_404.map(lambda req: (req[1], 1)).reduceByKey(lambda x, y: x + y)
sorted_errors_404_per_day = errors_404_per_day.sortBy(lambda req: datetime.strptime(req[0], '%d/%b/%Y'))
sorted_errors_404_per_day.collect()

[('01/Jul/1995', 316),
 ('02/Jul/1995', 291),
 ('03/Jul/1995', 474),
 ('04/Jul/1995', 359),
 ('05/Jul/1995', 497),
 ('06/Jul/1995', 640),
 ('07/Jul/1995', 570),
 ('08/Jul/1995', 302),
 ('09/Jul/1995', 348),
 ('10/Jul/1995', 398),
 ('11/Jul/1995', 471),
 ('12/Jul/1995', 471),
 ('13/Jul/1995', 532),
 ('14/Jul/1995', 413),
 ('15/Jul/1995', 254),
 ('16/Jul/1995', 257),
 ('17/Jul/1995', 406),
 ('18/Jul/1995', 465),
 ('19/Jul/1995', 639),
 ('20/Jul/1995', 428),
 ('21/Jul/1995', 334),
 ('22/Jul/1995', 192),
 ('23/Jul/1995', 233),
 ('24/Jul/1995', 328),
 ('25/Jul/1995', 461),
 ('26/Jul/1995', 336),
 ('27/Jul/1995', 336),
 ('28/Jul/1995', 94),
 ('01/Aug/1995', 243),
 ('03/Aug/1995', 304),
 ('04/Aug/1995', 346),
 ('05/Aug/1995', 236),
 ('06/Aug/1995', 373),
 ('07/Aug/1995', 537),
 ('08/Aug/1995', 391),
 ('09/Aug/1995', 279),
 ('10/Aug/1995', 315),
 ('11/Aug/1995', 263),
 ('12/Aug/1995', 196),
 ('13/Aug/1995', 216),
 ('14/Aug/1995', 287),
 ('15/Aug/1995', 327),
 ('16/Aug/1995', 259),
 ('17/Aug/19

5) O total de bytes retornados.

In [10]:
structured_rdd.map(lambda req: req[3]).reduce(lambda x, y: x + y)

65524314915